In [1]:
import pandas as pd

Pandas options to show all rows, columns (if required, use with caution with big tables)

* `pd.set_option('display.max_rows', None)`
* `pd.set_option('display.max_columns', None)`
* `pd.set_option('display.min_rows', 20)`

# Workbook 07

For this workbook, we will be using the ArcelorMittal shapes database downloaded from https://sections.arcelormittal.com/repo/Sections/Sections%20and%20Merchant%20Bars-ArcelorMittal_V2023-1.xlsx

In this workbook, you will be processing the data in the shapes database.

At the end of this workbook, you will be asked to create a module (i.e. copy/paste or move your code into a series of .py files). If you prefer, you can write your code and test functions directly in your module without needing to make the intermediary step of writing it in Jupyter first.

* Pros
    * You are working directly on the end-product
    * You are working directly in VS Code (if you prefer it)
* Cons
    * You cannot directly execute code from your module like you can in Jupyter in order to prototype and figure things out
    * You have to import your code from you module to run it in Jupyter (this is a normal thing to do, though)
    
It is simply your preference.

# Task A - Open and clean data

Open the "Sections and Merchant Bars-ArcelorMittal_V2023-1.xlsx" file in pandas and write a script that will "clean" or "tidy" the `DataFrame` for you. **Hint:** Refer to examples at the end of Lesson 07 for strategies on how to do this. Your cleaned DataFrame will represent all of the parallel flange "I-sections" in the .xlsx file (just the data on the **EN Sections** worksheet) with your choice of relevant columns. Once you have done this, use the `.to_csv()` method to save your DataFrame as a CSV file that will be accessible to your module. The data in the CSV file should not be altered in any way from the original Excel file. It should merely represent a selection, or subset, of the data in the Excel file. The key thing is that the raw data on disk should never change or be manipulated.

Design a function that takes no arguments and returns a DataFrame of the sections in your CSV file. All numeric values should have dtype of `float` (not `object`). Various numbers in the Excel file are _scaled_ for readability by some power of ten, e.g. the moments of inertia are typically scaled down into units of centimeters but the section dimensions are in units of millimeters. This is useful for readability but for calculation it can be advantageous for everything to be scaled equally (e.g. either all millimeteres or all centimeters, whichever you typically like to work in)

The information about the scaling is contained in the Excel sheet. In your function, after reading your data from the CSV file, apply the reverse scaling so that your DataFrame is ready to perform calculations with scaling removed from all columns (e.g. if Ix is listed in the Excel sheet as being in units of "cm<sup>4</sup>", then convert this value in to millimeters)

For your tests, create a "test file" that is a copy of your new CSV file but with only five rows of data. Write a test to verify that your "load dataframe" function actually contains the correct (unscaled) data after manually checking it with the Excel and CSV file.

## Task A Response



In [2]:
df = pd.read_excel("Sections and Merchant Bars-ArcelorMittal_V2023-1.xlsx", "EN sections", header=3, usecols= "B, D, F:M, AC:AN, AO:AR")
tidy = df.loc[~df['kg/m'].isna()]
renamed = tidy.rename(columns=
     {
         "Unnamed: 1": "Section name",
         "Pure bending yy": "Class flexural S355",
         "Unnamed: 41": "Class flexural S460",
         "Pure compression": "Class axial S355",
         "Unnamed: 43": "Class axial S460",
         " iz": "iz"
     })

for column in renamed.columns:
    if column != "Section name":
        pd.to_numeric(renamed[column], errors='coerce')

mask_IPE = renamed['Section name'].str.contains('IPE')
IPE_profiles = renamed.loc[mask_IPE].set_index('Section name')

display(IPE_profiles)
IPE_profiles.to_csv('IPE_profiles.csv')

,kg/m,h,b,tw,tf,r,hi,d,A,Iy,...,Wel.z,Wpl.z,iz,Ss,It,Iw,Class flexural S355,Class flexural S460,Class axial S355,Class axial S460
Section name,,,,,,,,,,,,,,,,,,,,,
IPE 750 x 220,220.0,779,266,16.5,30,20,719,679,280.7,279390,...,710,1114,5.7,9.9,620.5,13190,1,1,4,4
IPE 750 x 196,196.0,770,268,15.6,25.4,20,719,679,250.8,241470,...,610.2,960.1,5.6,8.9,416.5,11290,1,1,4,4
IPE 750 x 173,173.0,762,267,14.4,21.6,20,719,679,221.3,207010,...,515,811.1,5.5,8.1,278.7,9390,1,1,4,4
IPE 750 x 147,147.0,753,265,13.2,17,20,719,679,187.5,167250,...,399.3,632,5.2,7,163.2,7140,1,1,4,4
IPE 750 x 134,134.0,750,264,12,15.5,20,719,679,170.6,151890,...,361.2,569.6,5.2,6.6,124.9,6410,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IPE A 100,6.9,98,55,3.6,4.7,7,88.6,74.6,8.8,141.1,...,4.771,7.537,1.2,2.1,0.726,0.283,1,1,1,1
IPE AA 100,6.7,97.6,55,3.6,4.5,7,88.6,74.6,8.6,135.8,...,4.569,7.234,1.2,2,0.672,0.27,1,1,1,1
IPE 80,6.0,80,46,3.8,5.2,5,69.6,59.6,7.6,80.13,...,3.69,5.817,1,2,0.672,0.117,1,1,1,1


In [3]:
def read_profiles() -> pd.core.frame.DataFrame:
    """
    Returns a pandas dataframe with IPE-profiles with units scaled correctly.
    """
    profiles = pd.read_csv('IPE_profiles.csv')
    for column in profiles.columns:
        if column != "Section name":
            pd.to_numeric(profiles[column], errors='coerce')
            
    # Units applied over multiple columns by function
    def apply_unitscalar(columns, scalar):
        """
        Returns None, applies unitscalar to a list of columns
        """
        profiles[columns] = profiles[columns] * scalar
        return
    apply_unitscalar(['iy', 'iz', 'Ss'], 10)    
    
    # Units applied over multiple columns at once
    profiles[['A', 'Avz']] = profiles[['A', 'Avz']] * 100
    profiles[['Wel.y', 'Wpl.y', 'Wel.z', 'Wpl.z']] = profiles[['Wel.y', 'Wpl.y', 'Wel.z', 'Wpl.z']] * 1000
    
    # Units applied in a for loop per column
    for column in ['Iy', 'Iz', 'It']:
        profiles[column] = profiles[column] * 10000
    for column in ['Iw']:
        profiles[column] = profiles[column] * 1000000000
    
    return profiles

profiles = read_profiles()

In [4]:
from w_sections.processing import read_profiles

profiles = read_profiles('IPE_profiles.csv')
display(profiles)
print(profiles.columns)
print(profiles['h'].dtype)
print(type(profiles))
profiles['iy'].head()

,Section name,kg/m,h,b,tw,tf,r,hi,d,A,...,Wel.z,Wpl.z,iz,Ss,It,Iw,Class flexural S355,Class flexural S460,Class axial S355,Class axial S460
0,IPE 750 x 220,220.0,779.0,266,16.5,30.0,20,719.0,679.0,28070.0,...,710000.0,1114000.0,57.0,99.0,6205000.0,1.319000e+13,1,1,4,4
1,IPE 750 x 196,196.0,770.0,268,15.6,25.4,20,719.0,679.0,25080.0,...,610200.0,960100.0,56.0,89.0,4165000.0,1.129000e+13,1,1,4,4
2,IPE 750 x 173,173.0,762.0,267,14.4,21.6,20,719.0,679.0,22130.0,...,515000.0,811100.0,55.0,81.0,2787000.0,9.390000e+12,1,1,4,4
3,IPE 750 x 147,147.0,753.0,265,13.2,17.0,20,719.0,679.0,18750.0,...,399300.0,632000.0,52.0,70.0,1632000.0,7.140000e+12,1,1,4,4
4,IPE 750 x 134,134.0,750.0,264,12.0,15.5,20,719.0,679.0,17060.0,...,361200.0,569600.0,52.0,66.0,1249000.0,6.410000e+12,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,IPE A 100,6.9,98.0,55,3.6,4.7,7,88.6,74.6,880.0,...,4771.0,7537.0,12.0,21.0,7260.0,2.830000e+08,1,1,1,1
68,IPE AA 100,6.7,97.6,55,3.6,4.5,7,88.6,74.6,860.0,...,4569.0,7234.0,12.0,20.0,6720.0,2.700000e+08,1,1,1,1
69,IPE 80,6.0,80.0,46,3.8,5.2,5,69.6,59.6,760.0,...,3690.0,5817.0,10.0,20.0,6720.0,1.170000e+08,1,1,1,1
70,IPE A 80,5.0,78.0,46,3.3,4.2,5,69.6,59.6,640.0,...,2979.0,4692.0,10.0,17.0,3920.0,9.200000e+07,1,1,1,1


Index(['Section name', 'kg/m', 'h', 'b', 'tw', 'tf', 'r', 'hi', 'd', 'A', 'Iy',
       'Wel.y', 'Wpl.y', 'iy', 'Avz', 'Iz', 'Wel.z', 'Wpl.z', 'iz', 'Ss', 'It',
       'Iw', 'Class flexural S355', 'Class flexural S460', 'Class axial S355',
       'Class axial S460'],
      dtype='object')
float64
<class 'pandas.core.frame.DataFrame'>


0    314.0
1    309.0
2    305.0
3    297.0
4    297.0
Name: iy, dtype: float64

# Task B - Filter and Select Data

You are going to design _three small functions_ with tests:

* `sections_greater_than`
* `sections_less_than`
* `sort_by_weight`

### `sections_greater_than`

Your function will take the DataFrame and a series of keyword arguments describing target parameters of the section which should be greater than the target values. The first parameter will be the sections `DataFrame`. The second parameter will be `**kwargs`. 

Calling your function will look something like this:

```python
sections_greater_than(df, Ix=340e6) # Returns all sections with Ix greater or equal to this value
sections_greater_than(df, Iy=250e6, Zx=650e3) # Returns all sections with Iy >=, and Zx >=, the target values
```

What does this mean? Consider that the ArcelorMittal table has _many_ column headings and different measurements that apply for different kinds of sections. If we were to write a function that had optional keyword arguments for each one of those table columns that would be a lot of typing. 

Instead, we can have the function take `**kwargs` as an argument. This means that you, the user, can _make up_ your own function arguments and they will be available in the function as a dictionary. However, for the purpose of this function, your keyword arguments will match the _exact_ names of the columns in your DataFrame.

To get a feel for how using `**kwargs` works, try running, and playing around with, the below code.

```python
def practice_func(**kwargs):
    """
    Prints the keys and values in the kwargs dictionary
    """
    print(f"{kwargs=}")
    for k, v in kwargs.items():
        print(f"{k=}, {v=}")
        
practice_func(Ix=200, d=500)
practice_func(lamps="cardigans", hotels=3/4)
```

The implementation of your function will use `.loc` and boolean masking to return a new filtered `DataFrame` based on the given criteria.

### `sections_less_than`

This will be the exact same things `sections_greater_than` but will be for less-than-or-equal-to.

### `sort_by_weight`

Design a function called `sort_by_weight` that takes the sections DataFrame and returns a new DataFrame sorted by the linear weight of the sections in the table. Provide an optional keyword argument that allows the resulting DataFrame to be sorted in ascending or descending order.

In [5]:
def practice_func(**kwargs):
    """
    Prints the keys and values in the kwargs dictionary
    """
    print(f"{kwargs=}")
    for k, v in kwargs.items():
        print(f"{k=}, {v=}")
practice_func(Ix=200, d=500)
practice_func(lamps="cardigans", hotels=3/4)

kwargs={'Ix': 200, 'd': 500}
k='Ix', v=200
k='d', v=500
kwargs={'lamps': 'cardigans', 'hotels': 0.75}
k='lamps', v='cardigans'
k='hotels', v=0.75


## Task B Response

In [6]:
import numpy as np

def sections_greater_than(profiles: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """
    Returns a Dataframe with values greater than **kwargs based on a Dataframe
    """
    kw_masks = [profiles.loc[:, k] >= float(v) for k, v in kwargs.items()]
    mask = np.logical_and.reduce(kw_masks)    
    return profiles.loc[mask]

def sections_less_than(profiles: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """
    Returns a Dataframe with values less than **kwargs based on a Dataframe
    """
    kw_masks = [profiles.loc[:, k] <= float(v) for k, v in kwargs.items()]
    mask = np.logical_and.reduce(kw_masks)    
    return profiles.loc[mask]


def sort_by_weight(profiles: pd.DataFrame, ascending=True, **kwargs) -> pd.DataFrame:
    """
    Return a Dataframe sorted by weight
    """
    sorted_profiles = profiles.sort_values("kg/m", ascending=ascending)
    return sorted_profiles

from w_sections.processing import read_profiles

profiles = read_profiles('test_IPE_profiles.csv')
display(profiles.head(2))

print("Greater than", sections_greater_than(profiles, A=22000.0))
print("Less than   ", sections_less_than(profiles, A=19000.0))

print("Sorted by weight", sort_by_weight(sections_greater_than(profiles, A=22000.0)))

,Section name,kg/m,h,b,tw,tf,r,hi,d,A,...,Wel.z,Wpl.z,iz,Ss,It,Iw,Class flexural S355,Class flexural S460,Class axial S355,Class axial S460
0,IPE 750 x 220,220.0,779,266,16.5,30.0,20,719,679,28070.0,...,710000.0,1114000.0,57.0,99.0,6205000.0,13190000000000,1,1,4,4
1,IPE 750 x 196,196.0,770,268,15.6,25.4,20,719,679,25080.0,...,610200.0,960100.0,56.0,89.0,4165000.0,11290000000000,1,1,4,4


Greater than     Section name   kg/m    h    b    tw    tf   r   hi    d        A  ...  \
0  IPE 750 x 220  220.0  779  266  16.5  30.0  20  719  679  28070.0  ...   
1  IPE 750 x 196  196.0  770  268  15.6  25.4  20  719  679  25080.0  ...   
2  IPE 750 x 173  173.0  762  267  14.4  21.6  20  719  679  22130.0  ...   

      Wel.z      Wpl.z    iz    Ss         It              Iw  \
0  710000.0  1114000.0  57.0  99.0  6205000.0  13190000000000   
1  610200.0   960100.0  56.0  89.0  4165000.0  11290000000000   
2  515000.0   811100.0  55.0  81.0  2787000.0   9390000000000   

   Class flexural S355  Class flexural S460  Class axial S355  \
0                    1                    1                 4   
1                    1                    1                 4   
2                    1                    1                 4   

   Class axial S460  
0                 4  
1                 4  
2                 4  

[3 rows x 26 columns]
Less than        Section name   kg/m    h    

### Task B Tests

When you write tests for these functions, DO NOT use the whole ArcelorMittal database. Instead, create a small (like 4x4 dataframe) that allows you to test that your functions are working. The table can have whatever random data in it with whatever column names because you choose the column names you select when you call the function. You might be able to use the same little test table for each function.

Don't spend lots of time on this. Try and do it fast and easy. Copy-paste, etc.

In [7]:
# See separate test-file and use pytest to run

# Choose your path: 
* `handcalcs` - Task C(a)
* `SteelColumn` - Task C(b)
* `sectionproperties` - Task C(c)

# Task C(a) - _handcalcs_

Design three functions using the handcalcs `@handcalc` decorator: one for calculating the class of a section, one for calculating the $M_r$ of an I-section (assume lateral torsional buckling applies) and the other for calculating the $V_r$ of an I-section. Assume use of your module in Jupyter so `jupyter_display=True`.

Design a function that takes a `Series` (i.e. a row from the sections DataFrame) and an unbraced beam length, and returns a tuple representing the calculated class, $M_r$ and $V_r$ of the section, respectively. The function also takes an optional keyword parameter that determines whether or not the calculation of the class, $M_r$, and $V_r$ values should be displayed in Jupyter.

Design a function that takes the sections DataFrame (or a sub-set of the sections DataFrame) and adds four new columns to it: 
* Unbraced beam length
* **class**
* **Mr**
* **Vr**

The function also takes an optional keyword parameter that takes an integer. The integer represents the position of a row in the DataFrame and, if given, an example calculation of the class, $M_r$, and $V_r$ is displayed in Jupyter corresponding the data in the row.

Design a function that takes a DataFrame (same as above) and adds four new columns to it: 

* **Mf**
* **Vf**
* **DCR Moment** (demand/capacity ratio for the moment)
* **DCR Shear** (demand/capacity ratio for the shear)



# Task C(b) - _SteelColumn_

Using your "steel_columns" module from **Workbook_05**, design a function that takes a `Series` (i.e. a row from the sections DataFrame) and a column height (along with any other parameters you might need; you can choose to only use certain assumptions, e.g. only pin-pin ends), and returns a `SteelColumn` instance populated with the section data from the inputs.

Design a function that takes a `SteelColumn`, a dead load, and a live load, and returns a tuple representing the following data:

* factored load
* axial resistance
* demand/capacity ratio

Design a function that takes a `list[SteelColumn]`, a dead load, and a live load and returns a `list[tuple]`. The returned list of tuples contains tuples where each tuple represents the factored load, axial resistance, and demand/capacity ratio generated from each corresponding `SteelColumn`

Design a function that takes a DataFrame (i.e. the whole sections table or a sub-set of the sections table) a column height (along with any of the other parameters you needed in the previous function), a steel yield strength, a dead load and a live load, and returns the DataFrame with five new columns added: 
* **Height**
* **Dead**
* **Live**
* **Factored load**
* **Axial Resistance**
* **DCR** (demand/capacity ratio)

In [8]:
# See implementation in the module folder

# Task C(c) - _sectionproperties_

Design a function that takes a `Series` (i.e. a row from the sections DataFrame) and returns a sectionproperties `Section` object ready for analysis. Assume material parameters that are relevant in your market for I-sections.

Design a function that takes a `Section` and a set of force actions (e.g. $N$, $M_x$, $M_y$, $V_x$, $V_y$, $M_z$, which could all be optional keyword parameters set to `0.0`) and returns the maximum Von Mises stress that occurs in the section. This will require you to become a bit familiar with the `StressPost` class in `sectionproperties`. 

> Documentation is here: https://sectionproperties.readthedocs.io/en/latest/rst/api.html#stresspost-class
>
> Basically, when you run a stress analysis, a `StressPost` object is returned. It has a method called `.get_stress()` which returns a dictionary of the stress components. Use that dictionary to get the max Von Mises stress.

Design a function that takes the sections DataFrame (or a subset of the sections DataFrame), the set of force actions, and a steel yield strength and returns a new DataFrame represnadds _nine columns_ to the Da:

* Steel yield strength 
* **N**
* **Mx**
* **My**
* **Vx**
* **Vy**
* **Mz**
* **sig_vm Max** (Maximum von Mises stress)
* **DCR stress**



# Task D

Whichever path you selected for Task C, it is time to transfer your functions into your module. Call your module `w_sections` and transfer all of your code and all of your imports into it. 



# Task E

Make up a simple structural problem for yourself where you import the functions you need from your module and use your functions to get some results.

e.g. (If path C(c) was taken...) "I am going to say that I need to select a section that needs to be between 12 and 14 inches deep, 4 to 7 inches wide, that is subjected to axial, shear and moment in two directions, and a leeeeetle bit of torsion. I will use my tools to select a section where the von Mises stress does not exceed my factored capacity."

No need to write tests for this! Just use your stuff and have fun!

In [11]:
from w_sections.processing import read_profiles, calculate_steelcolumns, sort_by_weight, sections_less_than

profiles = read_profiles('IPE_profiles.csv')
fy = 235 # MPa
column_height = 6000 # mm
deadload = 40000 # N
liveload = 50000 # N
checked_profiles = calculate_steelcolumns(profiles, column_height, fy, deadload, liveload)
drop_failed = sections_less_than(checked_profiles, DCR=1)

print(f"A IPE column with heigh {column_height/1000} m and load of {deadload/1000} & {liveload/1000} kN\n")

# Best profile by weight
best_by_weight = sort_by_weight(drop_failed)
by_weight_row_nr = best_by_weight[['kg/m']].idxmin()
by_weight_row = best_by_weight.loc[by_weight_row_nr]
print(f"The best profile by weight is {by_weight_row['Section name'].values[0]}, with an DCR of {by_weight_row.DCR.values[0]:.3f}")

# Best profile by DCR
best_by_dcr = drop_failed.sort_values("DCR", ascending=False)
by_dcr_row_nr = best_by_dcr[['DCR']].idxmax()
by_dcr_row = best_by_dcr.loc[by_dcr_row_nr]
print(f"The best profile by DCR is {by_dcr_row['Section name'].values[0]}, with an DCR of {by_dcr_row.DCR.values[0]:.3f}")

# Show tasble
print("\nThe top 5 results by DCR are:")
display(best_by_dcr.head())

A IPE column with heigh 6.0 m and load of 40.0 & 50.0 kN

The best profile by weight is IPE AA 140, with an DCR of 0.792
The best profile by DCR is IPE 120, with an DCR of 0.931

The top 5 results by DCR are:


,Section name,kg/m,h,b,tw,tf,r,hi,d,A,...,Class flexural S355,Class flexural S460,Class axial S355,Class axial S460,Height,Dead,Live,Factored load,Axial Resistance,DCR
63,IPE 120,10.4,120.0,64,4.4,6.3,7,107.4,93.4,1320.0,...,1,1,1,1,6000,40000,50000,123000.0,132078.476130,0.931265
62,IPE AA 140,10.1,136.6,73,3.8,5.2,7,126.2,112.2,1280.0,...,1,1,2,3,6000,40000,50000,123000.0,155242.601433,0.792308
61,IPE A 140,10.5,137.4,73,3.8,5.6,7,126.2,112.2,1340.0,...,1,1,2,3,6000,40000,50000,123000.0,164516.944344,0.747643
60,IPE 140,12.9,140.0,73,4.7,6.9,7,126.2,112.2,1640.0,...,1,1,1,2,6000,40000,50000,123000.0,203488.445292,0.604457
59,IPE AA 160,12.3,156.4,82,4.0,5.6,9,145.2,127.2,1570.0,...,1,1,3,4,6000,40000,50000,123000.0,223528.525642,0.550265
